# Lecture 2: Advanced Python
## ECON5170 Computational Methods in Economics
#### Author: Zhentao Shi
#### Date: March 2020

# Advanced R (Python)


### Introduction

In this lecture, we will talk about efficient computation in R (Python).

*  **R is a vector-oriented language. In most cases, vectorization speeds up computation**.
*  We turn to more CPUs for parallel execution to save time if there is no more room to optimize the code to improve the speed.
*  Clusters are accessed remotely. Communicating with a remote cluster is different from operating a local machine.

### Vectorization

Despite mathematical equivalence, various ways of calculation can perform distinctively in terms of computational speed.

Does computational speed matter?
For a job that takes less than a minutes, the time difference is not a big deal.
For modern economic structural estimation problems commonly seen in industrial organization, a single estimation can take up to a week. For those problems code optimization is essential.

Other computational intensive procedures include bootstrap, simulated maximum likelihood and simulated method of moments. Even if a single execution does not take much time, repeating such a procedure for thousands of replications will consume a non-trivial duration.

Of course, optimizing code takes human time. It is a balance of human time and machine time.

__Example__

The heteroskedastic-robust variance for the OLS regression is
$$(X'X)^{-1} X'\hat{e}\hat {e}'X (X'X)^{-1}$$
The difficult part is $X'\hat{e}\hat {e}'X=\sum_{i=1}^n \hat{e}_i^2 x_i x_i'$.
There are at least 4 mathematically equivalent ways to compute this term.

1.  literally sum over $i=1,\dots,n$ one by one.
2.  $X' \mathrm{diag}(\hat{e}^2) X$, with a dense central matrix.
3.  $X' \mathrm{diag}(\hat{e}^2) X$, with a sparse central matrix.
4.  Do cross product to `X*e_hat`. It takes advantage of the element-by-element operation.

We first generate the data of binary response and regressors. Due to the discrete nature of the dependent variable, the error term in the linear probability model is heteroskedastic. It is necessary to use the heteroskedastic-robust variance to consistently estimate the asymptotic variance of the OLS estimator. The code chunk below estimates the coefficients and obtains the residual.

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix 
import random
import datetime
import math
import statistics
import matplotlib.pyplot as plt

In [3]:
def lpm(n):
    # estimation in a linear probability model
    # set the parameters
    b0 = np.array([[-1],[1]])
    # generate the data
    e = np.random.normal(size = (n,1))
    X = np.ones((n,2))
    X[:, 1] = np.random.normal(n)
    Y = X @ b0 + e
    # note that in this regression b0 is not converge to b0 because the model is changed.
    
    # OLS estimation
    bhat = np.linalg.inv( X.T @ X) @ X.T @ Y
    
    e_hat = Y - X @ bhat
    return(X, e_hat)

We run the 4 estimators for the same data, and compare the time.

In [18]:
# an example of robust variance matrix.
# compare the implementation via matrix and vectorization.

# n = 5000; Rep = 10 # large matrix

n = 50; Rep = 1000 # small matrix

for opt in range(0, 4):
    
    pts0 = datetime.datetime.now()
    
    for iter in range(Rep):
        
        np.random.seed(iter) # to make sure that the data used different estimation methods are the same 
    
        [X, e_hat] = lpm(n)
        XXe2 = np.zeros((2,2))
        
        if opt == 0: # element by element
            for i in range(len(X)):
                XXe2 =  XXe2 + e_hat[i]**2 * X[i,] @ X[i,].T
                
        elif opt == 1: # the vectorized version
            e_hatt2 = np.diag( np.square(e_hat).flatten() )            
            XXe2 = X.T @ e_hatt2 @ X
            
        elif opt == 2: # the vectorized version with sparse matrix
            e_hat2_M = csr_matrix( np.diag( np.square(e_hat).flatten() ) )
            XXe2 = X.T @ e_hatt2 @ X
        
        elif opt == 3: # the best vectorization method. No waste
            Xe = X * e_hat
            XXe2 = Xe.T @ Xe
            
        XX_inv = np.linalg.inv( X.T @ X )
        sig_B =  XX_inv @ XXe2 @ XX_inv
    
    print("n = ", n, ", Rep = ", Rep, ", opt = ", opt, ", time = ", datetime.datetime.now() - pts0, "\n")

n =  50 , Rep =  1000 , opt =  0 , time =  0:00:00.242816 

n =  50 , Rep =  1000 , opt =  1 , time =  0:00:00.066821 

n =  50 , Rep =  1000 , opt =  2 , time =  0:00:00.199980 

n =  50 , Rep =  1000 , opt =  3 , time =  0:00:00.045877 



**Results with n = 50 and Rep = 1000**

n =  50 , Rep =  1000 , opt =  0 , time =  0:00:00.321856 

n =  50 , Rep =  1000 , opt =  1 , time =  0:00:00.061392 

n =  50 , Rep =  1000 , opt =  2 , time =  0:00:33.542949 

n =  50 , Rep =  1000 , opt =  3 , time =  0:00:00.046890 

We clearly see the difference in running time, though the 4 methods are mathematically the same.
When $n$ is small, `matrix` is fast and `Matrix` is slow; the vectorized version is the fastest.
When $n$ is big, `matrix` is slow and `Matrix` is fast; the vectorized version is still the fastest.

## Efficient Loop

In standard `for` loops, we have to do a lot of housekeeping work. 

In [19]:
def CI(x): # construct confidence interval
           # x is a vector of random variables
    n = len(x)
    mu = np.mean(x)
    sig = np.std(x)
    upper = mu + 1.96 / np.sqrt(n) * sig
    lower = mu - 1.96 / np.sqrt(n) * sig
    return {'lower': lower, 'upper': upper}

This is a standard `for` loop.

In [29]:
Rep = 100000
sample_size = 10
mu = 2

# append a new outcome after each loop
pts0 = datetime.datetime.now() # check time
for i in range(Rep):
    x = np.random.poisson(mu, sample_size)
    bounds = CI(x)
    out_i = ( ( bounds['lower'] <= mu  ) & (mu <= bounds['upper']) )
    if i == 0:
        out = np.array(out_i)
    else:
        out = np.append(out, out_i)

stat_cover = np.count_nonzero(out)/Rep*100

print( "empirical coverage probability = ", stat_cover, "% \n") # empirical size
pts1 = datetime.datetime.now() - pts0 # check time elapse
print(pts1, "\n")

empirical coverage probability =  89.13499999999999 % 

0:00:03.463812 



### Classical loop with an empty list

In [30]:
Rep = 100000
sample_size = 10
mu = 2

# append a new outcome after each loop

pts0 = datetime.datetime.now() # check time

# Empty list
out = list()

for i in range(Rep):    
    x = np.random.poisson(mu, sample_size)
    bounds = CI(x)
    out.append( ( bounds['lower'] <= mu  ) & (mu <= bounds['upper']) )

stat_cover = out.count(True)/Rep*100


print( "empirical coverage probability = ", stat_cover, "% \n") # empirical size
pts1 = datetime.datetime.now() - pts0 # check time elapse
print(pts1) 

empirical coverage probability =  89.22099999999999 % 

0:00:02.937388


### Classical loop with an existing list and overwriting

In [31]:
Rep = 100000
sample_size = 10
mu = 2

# override an existing list

pts0 = datetime.datetime.now() # check time

# List with same length as Rep
out = [0] * Rep

for i in range(Rep):
    x = np.random.poisson(mu, sample_size)
    bounds = CI(x)
    out.append((bounds['lower'] <= mu  ) & (mu <= bounds['upper']))

stat_cover = out.count(True)/Rep*100

print( "empirical coverage probability = ", stat_cover, "% \n") # empirical size
pts1 = datetime.datetime.now() - pts0 # check time elapse
print(pts1) 

empirical coverage probability =  89.208 % 

0:00:02.849054


Pay attention to the line `out = [0] * Rep`. It *pre-defines* a vector `out` to be filled by `out[i] = out.append((bounds['lower'] <= mu  ) & (mu <= bounds['upper']))`. The computer opens a continuous patch of memory for the vector `out`. When new result comes in, the old element is replaced. If we do not pre-define `out` but append one more element in each loop, the length of `out` will change in each replication and every time a new patch of memory will be assigned to store it. The latter approach will spend much more time just to locate the vector in the memory.

`out` is the result container. In a `for` loop, we pre-define a container, and replace the elements
of the container in each loop by explicitly calling the index.

### For loop with a function

In [33]:
Rep = 100000
sample_size = 10
mu = 2

# Create a function and let it run with a for loop

def capture(i):
    x = np.random.poisson(mu, sample_size)
    bounds = CI(x)
    return ((bounds['lower'] <= mu) & (mu <= bounds['upper']))

pts0 = datetime.datetime.now()  # check time

out = [capture(i) for i in range(Rep)]

stat_cover = out.count(True)/Rep*100

print( "empirical coverage probability = ", stat_cover, "% \n") # empirical size
pts1 = datetime.datetime.now() - pts0  # check time elapse
print(pts1)

empirical coverage probability =  89.208 % 

0:00:02.766142


### Apply() has still some error

In [10]:
Rep = 10
sample_size = 1000
mu = 2

# Create a function and let it run with apply

def capture(i):
    x = np.random.poisson(mu, sample_size)
    bounds = CI(x)
    return ((bounds['lower'] <= mu) & (mu <= bounds['upper']))

pts0 = datetime.datetime.now()  # check time

### apply ###
from apply import apply


out = [apply(capture, args=(i)) for x in range(Rep)]
out = list(out)
print("empirical coverage probability = ", out.count(True) / Rep * 100, "\n")  # empirical size
pts1 = datetime.datetime.now() - pts0  # check time elapse

NameError: name 'i' is not defined

In [115]:
Rep = 10
sample_size = 1000
mu = 2

# Create a function and let it run with map

def capture(i):
    x = np.random.poisson(mu, sample_size)
    bounds = CI(x)
    return ((bounds['lower'] <= mu) & (mu <= bounds['upper']))

pts0 = datetime.datetime.now()  # check time

out = map(capture, range(Rep), )
out = list(out)
stat_cover = out.count(True)/Rep*100

print( "empirical coverage probability = ", stat_cover, "% \n") # empirical size

pts1 = datetime.datetime.now() - pts0  # check time elapse
print(list(out))

empirical coverage probability =  80.0 % 

[True, True, True, True, True, True, False, True, True, False]


## Parallel Computing

Parallel computing becomes essential when the data size is beyond the storage of a single computer, for example  {% cite li2018embracing %}.
Here we explore the speed gain of parallel computing on a multicore machine.

Here we explore the speed gain of parallel computing on a multicore machine.

The package `multiprocessing` is the choice for parallel computing in Python.
Below is the basic structure. 

In [4]:
# import multiprocessing
from multiprocessing import Process, current_process
import multiprocessing as mp
import os

print("Number of processors: ", mp.cpu_count())

Number of processors:  4


In [5]:
Rep = 10
sample_size = 10
mu = 2

for i in range(Rep):
    np.random.seed(i)
    x = np.random.poisson(mu, sample_size)
    print(x)

[3 2 5 1 0 0 7 1 3 3]
[2 1 0 1 2 2 0 3 3 3]
[1 2 1 2 2 1 4 1 1 0]
[2 3 1 1 2 2 2 2 1 2]
[5 1 1 2 2 0 1 5 0 1]
[2 4 1 0 2 2 2 2 1 1]
[3 0 2 2 5 3 5 2 3 4]
[0 4 1 1 3 2 2 2 2 3]
[4 0 2 3 3 2 0 1 1 1]
[0 2 1 1 0 1 2 4 4 3]


If we have two CPUs running simultaneously, in theory we can cut the time to a half of that on a single CPU. Is that what happening in practice?

### Multiprocessing with the `process` class

In [121]:
Rep = 10
sample_size = 1000
mu = 2


def capture(i,return_dict):
    np.random.seed(i)
    x = np.random.poisson(mu, sample_size)
    bounds = CI(x)
    result = ((bounds['lower'] <= mu) & (mu <= bounds['upper']))
    # process_id = os.getpid()
    # print("Process ID: " + str(process_id))
    print("The result: " + str(result))
    return_dict[i]=result

pts0 = datetime.datetime.now()  # check time

manager = mp.Manager()
return_dict = manager.dict()
jobs = []

for i in range(Rep):
    p = Process(target=capture, args=(i,return_dict))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

# print(return_dict.values())

stat_cover = jobs.count(True)/Rep*100

print( "empirical coverage probability = ", stat_cover, "% \n") # empirical size
pts1 = datetime.datetime.now() - pts0 # check time elapse
print("The the calculation time is:", pts1, "\n")

The result: True
The result: True
The result: False
The result: True
The result: True
The result: True
The result: True
The result: False
The result: False
The result: True
empirical coverage probability =  0.0 % 

The the calculation time is: 0:00:00.284145 



### Multiprocessing with the `pool` class & `apply()`

In [122]:
Rep = 200
sample_size = 2000
mu = 2


pts0 = datetime.datetime.now()  # check time

def capture(i):
    x = np.random.poisson(mu, sample_size)
    bounds = CI(x)
    return ((bounds['lower'] <= mu) & (mu <= bounds['upper']))

# Only allows to run 4 processes at the time
pool = mp.Pool(processes=4)

# Initiate the multiprocess process wit apply()
results = [pool.apply(capture, args=(i,)) for x in range(Rep)]

print( "empirical coverage probability = ", results.count(True)/Rep*100, "\n") # empirical size
pts1 = datetime.datetime.now() - pts0 # check time elapse
print("The the calculation time is:", pts1, "\n")
# print(results) 

empirical coverage probability =  48.0 

The the calculation time is: 0:00:01.718949 



### Multiprocessing with the `pool` class & `map()`

In [123]:
Rep = 200
sample_size = 2000
mu = 2

pts0 = datetime.datetime.now()  # check time
def capture(i):
    x = np.random.poisson(mu, sample_size)
    bounds = CI(x)
    return ((bounds['lower'] <= mu) & (mu <= bounds['upper']))
    
# Only allows to run 4 processes at the time
pool = mp.Pool(processes=4)

# Initiate the multiprocess process with the map()
results = pool.map(capture, range(Rep), )

print( "empirical coverage probability = ", results.count(True)/Rep*100, "\n") # empirical size
pts1 = datetime.datetime.now() - pts0 # check time elapse
print("The the calculation time is:", pts1, "\n")
# print(results) 

empirical coverage probability =  50.0 

The the calculation time is: 0:00:00.975129 



Process ForkPoolWorker-210:
Process ForkPoolWorker-212:
Process ForkPoolWorker-211:
Process ForkPoolWorker-209:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/marckullmann/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/marckullmann/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/marckullmann/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/marckullmann/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/marckullmann/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/marckullmann/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args

## Remote Computing

Investing money from our own pocket to an extremely powerful laptop to conduct heavy-lifting computational work
is unnecessary. (i) We do not run these long jobs every day, it is more cost efficient
to share a workhorse. (ii) We cannot keep our laptop always on when we move it
around. The right solution is remote computing on a server.

No fundamental difference lies between local and remote computing.
We prepare the data and code, open a shell for communication, run the code, and collect the results.
One potential obstacle is dealing with a command-line-based operation system.
Such command line tools is the norm of life two or three decades ago, but today we mostly
work in a graphic operating system like Windows or OSX.
For Windows users (I am one of them), I recommend [PuTTY](http://www.putty.org/), a shell, and [WinSCP](http://winscp.net/eng/download.php), a graphic interface for input and output.

Most servers in the world are running Unix/Linux operation system.
Here are a few commands for basic operations.

* mkdir
* cd
* copy
* top
* screen
* ssh user@address
* start a program

Our department's computation infrastructure has been improving.
A server dedicated to  professors is a 16-core machine. I have opened an account for you.
You can try out this script on `econsuper`.

1. Log in `econsuper.econ.cuhk.edu.hk`;
2. Save the code block below as `loop_server.R`, and upload it to the server;
3. In a shell, run `R --vanilla <loop_server.R> result_your_name.out`;
4. To run a command in the background, add `&` at the end of the above command. To keep it running after closing the console, add `nohup` at the beginning of the command.